# Kevin Bethune: CCO/CEO of dreams • design + life

### "We should leverage uncertainty for the powerful, creativity-inducing variable that it is." —via [Reimagining Design: Unlocking Strategic Innovation](https://mitpress.mit.edu/featured-book-reimagining-design/)

### With Kevin, we use embedding models to collect relevant quotes from his past to guide how career advice may happen.

Instructions for getting cozy with this AI recipe are on the [GitHub page's README](https://aka.ms/CAIK-repo). 

# 🧑‍🍳 Recipe for AI-driven Career Advising

- ~500 tokens from COMPLETION Pre-trained Foundation Model
- ~1000 tokens from EMBEDDING Pre-trained Foundation Model
- 1 list of Reimagining Design quotes
- 1 PLUGIN for career advice
 
First, run the list of quotes through the EMBEDDING model. 

Formulate which expertise you want to draw upon from the list of quotes.

Lastly, take that context and have it drive the Career Advice plugin.

Taste for quality, and then plate appropriately.

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather the core tools 🧰

When running the following cell, if asked to "select your kernel" (note this will be referring to the Jupyter notebook's kernel and not Semantic Kernel) then choose `.NET Interactive` from the available menu options.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import ../config/Settings.cs 

#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta3"
#r "nuget: System.Linq.Async, 6.0.1"

# Fire up a kernel with TWO models 🔥

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var (memoryBuilder, kernelBuilder) = (new MemoryBuilder(), new KernelBuilder());  

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
    kernelBuilder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
    kernelBuilder.WithOpenAIChatCompletionService(model, apiKey, orgId);
}

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());

var (kernel, memory) = (kernelBuilder.Build(), memoryBuilder.Build());  

# Generate embeddings for later 🧲 gathering

In [ ]:
const string MemoryCollectionName = "kevinQuotes";

// Kevin quoted by his fans on GoodReads: https://www.goodreads.com/author/quotes/21599683.Kevin_G_Bethune
string[] kevinQuotes = {   
    "I believe it’s very important to be more up-front about the hard realities we will face as innovators and still push ourselves forward with a sense of optimism and courage.",  
    "The team needs to learn how to work together, align around the right imperatives, and develop their unique chemistry.",  
    "The constellation of insights must line up in a compelling way, and usually those insights need to come from different vantage points to ensure an idea has a shot of making it into a well-timed, successful innovation that is desirable, viable, and technically feasible.",  
    "We should leverage uncertainty for the powerful, creativity-inducing variable that it is.",  
    "The quest for innovation against newly emerging realities requires us to bring disparate teams closer together and to create more bandwidth for actually collaborating with one another.",  
    "Design still fights the perception of being the very last step in the value stream.",  
    "Thanks to design thinking, most organizations are beginning to see the value of embracing design earlier in their processes.",  
    "Moving on to incubation, as we build out our designed experiences, the designer should no longer (and can no longer) work in isolation from other disciplines.",  
    "Design will play an important role in stringing the collective team’s efforts together in a cohesive way to optimize impact in the marketplace."  
};

for (int i = 0; i < kevinQuotes.Length; i++)
{
    await memory.SaveInformationAsync(MemoryCollectionName, id: $"kev{i}", text: kevinQuotes[i]);
}

# Experiment 🧪 with what you think is relevant and important 

In [ ]:
var questions = new[]
{
    "create a team",
    "challenges of businesses"
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine($"{q}\n> {response?.Metadata.Text}");
}

# Plate the partially finished meal 🍽️

In [ ]:
using Microsoft.AspNetCore.Html;
using Markdig;

String foundInMemory = "";

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
   foundInMemory += $"## {q}\n> {response?.Metadata.Text}\n\n";
}

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml(foundInMemory, myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent

# Now let's use a ✨ COMPLETION model that's grounded

In [ ]:
var pluginsSKDirectory = "plugins-sk";

var question = "What should I become?";

var mnFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsSKDirectory, "Mentor");

var result = await kernel.RunAsync(question, mnFunctions["CareerAdviceGeneric"]);
var resultString = result.GetValue<string>();

Console.WriteLine($"Question: {question}\nNot-Kevin-ish Advice: {resultString}");


That was totally made up. Right? Let's now get the model grounded in things that Kevin actually believes in :+).

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

var myContext = new ContextVariables(); 

var response = await memory.SearchAsync(MemoryCollectionName, question).FirstOrDefaultAsync();
var belief = response?.Metadata.Text;

Console.WriteLine($"Question: {question}\nBelief: {belief}");

myContext.Set("input", question); 
myContext.Set("beliefs", belief); 

var myResult = await kernel.RunAsync(myContext,mnFunctions["CareerAdviceGrounded"]);
var resultString = result.GetValue<string>();

Console.WriteLine($"---\nKevin-ish Advice: {myResult}");

# Let's look at the science 📐

In [ ]:
using System.Text.Json;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI;

var usageresult = myResult.FunctionResults.SelectMany(l => l.GetModelResults() ?? Enumerable.Empty<ModelResult>()).LastOrDefault()?.GetOpenAIChatResult()?.Usage;
var injson = JsonSerializer.Serialize(usageresult, new JsonSerializerOptions { WriteIndented = true });
Console.WriteLine($"🧱 The MODEL used:\n\n> {model}\n");
Console.WriteLine($"💸 How many TOKENS were used?\n\n{injson}\n");
String functionName = "CareerAdviceGrounded";

var reqjson = JsonSerializer.Serialize(mnFunctions[functionName].RequestSettings, new JsonSerializerOptions { WriteIndented = true });
var doc = JsonDocument.Parse(reqjson);
double temperatureValue = doc.RootElement.GetProperty("temperature").GetDouble();
double topPValue = doc.RootElement.GetProperty("top_p").GetDouble();

Console.WriteLine($"🌡️ What TEMPERATURE setting was used?\n\n> {temperatureValue} (0 is 🥶 ; 1 is 🥵)\n");
Console.WriteLine($"🙊 What TOP P setting was used?\n\n> {topPValue} (0 is 😑 ; 1 is 🤯)");